In [23]:
import asyncio
import websockets
import json
import pandas as pd
apiKeys=pd.read_csv("/Users/maier-borst.h/Documents/apiKeys.csv", sep=";")
apiKeyAIS=apiKeys[apiKeys["Service"]=="AISStream"]["key"].iloc[0]
apiKeyAIS
from datetime import datetime, timezone

async def connect_ais_stream():

    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {"APIKey": apiKeyAIS,  # Required !
                             "BoundingBoxes": [[[-120, -10], [0, 10]]], # Required!
                             "FilterMessageTypes": ["PositionReport"]} # Optional!

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)
        print(websocket)
        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            websocket.close();

            if message_type == "PositionReport":
                # the message parameter contains a key of the message type which contains the message itself
                ais_message = message['Message']['PositionReport']
                print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Latitude: {ais_message['Longitude']}")



In [6]:
await connect_ais_stream()

/var/folders/55/79cxhkf56b38c317w6g5zkr0mmx5lp/T/ipykernel_30993/2793776042.py:23: RuntimeWarning: coroutine 'WebSocketCommonProtocol.close' was never awaited
  websocket.close();


[2024-03-12 10:09:24.060136+00:00] ShipId: 367480140 Latitude: 0 Latitude: 0
[2024-03-12 10:09:24.271096+00:00] ShipId: 303308000 Latitude: 0 Latitude: 0


CancelledError: 

In [41]:
async def connect_ais_stream():
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": apiKeyAIS,  # Required !
            "BoundingBoxes": [[[41.97, 12.53], [43.66, 13.83]]],  # Required!
            "FilterMessageTypes": ["PositionReport"]  # Optional!
        }

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        try:
            await asyncio.wait_for(process_messages(websocket), timeout=120)
        except asyncio.TimeoutError:
            print("Timeout occurred. Stopping execution.")

async def process_messages(websocket):
    async for message_json in websocket:
        message = json.loads(message_json)
        message_type = message["MessageType"]
        websocket.close()

        ais_message = message['Message']
        print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Longitude: {ais_message['Longitude']}")

# Call the function
#asyncio.run(connect_ais_stream())

In [39]:
#need this for script not in pynb asyncio.run(connect_ais_stream())


In [26]:
counter = 1
async def connect_ais_stream_ai(counter):
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": apiKeyAIS,  # Required !
            "BoundingBoxes": [[[-120, -10], [0, 10]]],  # Required!
            "FilterMessageTypes": ["PositionReport"]  # Optional!
        }

        subscribe_message_json = json.dumps(subscribe_message)
        while(counter<10):
            await websocket.send(subscribe_message_json)
    
            try:
                await asyncio.wait_for(process_messages(websocket, counter), timeout=30)
            except asyncio.TimeoutError:
                print("Timeout occurred. Stopping execution.")
        print("Done")
async def process_messages(websocket, counter):
    async for message_json in websocket:
        message = json.loads(message_json)
        try:
            message_type = message["MessageType"]
            websocket.close()
    
            if message_type == "PositionReport":
                print(counter)
                ais_message = message['Message']['PositionReport']
                print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Latitude: {ais_message['Longitude']}")
                counter=counter+1
            
# Call the function


In [42]:
await(connect_ais_stream())

Timeout occurred. Stopping execution.
